<a href="https://colab.research.google.com/github/dvlpana/SparkPlatzi/blob/main/CursoSpakPlatzi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!apt-get install  openjdk-8-jdk-headless -qq > /dev/null
!ad-apt-repository ppa:openjdk-r/ppa
!apt-get update && sudo apt-get upgrade -y
!apt-get -y install openjdk-8-jre
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar -xvf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

/bin/bash: ad-apt-repository: command not found
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InReleas

In [ ]:
!pip install -q findspark

In [ ]:
!ls

In [ ]:
import time
print("Sleeping")
time.sleep(30) # sleep for a while; interrupt me!
print("Done Sleeping")

In [ ]:
!ls

In [ ]:
!head -n 15 /content/sample_data/files_curso_spark/data.csv

In [ ]:
!spark-submit /content/sample_data/files_curso_spark/codeExample.py /content/sample_data/files_curso_spark/data.csv

In [ ]:
!ls

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark


In [ ]:
archivo = './sample_data/files_curso_spark/data.csv'
df_spark = spark.read.csv(archivo, inferSchema=True, header=True)

# imprimir tipo de archivo
print(type(df_spark))

¿Nombre de las Columnas de dataframe?

In [ ]:
df_spark.columns

Estructura del dataframe

In [ ]:
df_spark.printSchema()

Ver los primeros 20 registros del dataframe

In [ ]:
df_spark.show()

Descricipcion Estadistica del dataframe

In [ ]:
df_spark.describe().toPandas().transpose()

Descripcion estadistica de una sola columna (‘median_house_value’)

In [ ]:
df_spark.describe(['color']).show()

## 8 Transformaciones y Acciones

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

sc = spark.sparkContext
lines = sc.textFile("spark-2.4.4-bin-hadoop2.7/README.md")
sc



In [ ]:
path = '/content/sample_data/files_curso_spark/'
equiposOlimpicosRDD = sc.textFile(path+'paises.csv') \
.map(lambda line: line.split(","))

# Muestra la data con take , evita utilizar collect()

In [ ]:
equiposOlimpicosRDD.take(10)



9 Acciones de modificacion sobre RDDs

In [ ]:
##Contar cuantas siglas tenemos
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

In [ ]:
##Agrupación a partir de siglas y obtener cantidad de equipos por país
equiposOlimpicosRDD \
.map(lambda x: (x[2], x[1])) \
.groupByKey() \
.mapValues(len).take(5)

In [ ]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

In [ ]:
##Agrupación a partir de siglas y obtener cantidad de equipos por país
equiposOlimpicosRDD \
.map(lambda x: (x[2], x[1])) \
.groupByKey() \
.mapValues(list).take(3)

In [ ]:
##Obtener los datos sólo de un país o contenido con filter()
equiposArgentinos = equiposOlimpicosRDD.filter(lambda x: "ARG" in x)
equiposArgentinos.collect()

In [ ]:
## Cuenta todos los valores, nada recomendable para datos extensos
equiposOlimpicosRDD.count()

In [ ]:
##Cuenta  hasta el tiempo aproximado
equiposOlimpicosRDD.countApprox(20)

10 Acciones de conteo sobre RDDs

In [ ]:
## importamos ambos
deportistaOlimpicoRDD = sc.textFile(path+'deportista.csv') \
.map(lambda line: line.split(","))

deportistaOlimpicoRDD2 = sc.textFile(path+'deportista2.csv') \
.map(lambda line: line.split(","))

In [ ]:
## Unimos los RDD con union(). Spark solo tiene esta operacion
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [ ]:
deportistaOlimpicoRDD.count()

In [ ]:
##Ver contenido en equipos olimpicos
equiposOlimpicosRDD.top(2)

In [ ]:
deportistaOlimpicoRDD.top(2)

In [ ]:
# seleccionamos las comumnas ID para hacer un join
deportistaOlimpicoRDD \
.map(lambda deportista: [deportista[-1],deportista[:-1]]) \
.join(equiposOlimpicosRDD.map(lambda equipo: [equipo[0], equipo [2]])) \
.take(6)

In [ ]:
# utilizamos takeSample (repetidos, muestra, semilla)
deportistaOlimpicoRDD \
.map(lambda deportista: [deportista[-1],deportista[:-1]]) \
.join(equiposOlimpicosRDD.map(lambda equipo: [equipo[0], equipo [2]])) \
.takeSample(False,6,25)

In [ ]:
resultado = sc.textFile(path+'resultados.csv') \
.map(lambda line: line.split(","))

In [ ]:
resultadoGanador = resultado.filter(lambda w: 'NA' not in w[1])

In [ ]:
resultadoGanador.take(5)

In [ ]:
##Reto Hacer un join con equipos y deportista para obtener valores importantes
deportistaOlimpicoRDD \
.map(lambda deportista: [deportista[-1],deportista[:-1]]) \
.join(equiposOlimpicosRDD.map(lambda equipo: [equipo[0], equipo [2]])) \
.join(resultadoGanador.map(lambda resultado: [resultado[2], resultado [1]])) \
.takeSample(False, 6, 25)


Clase 11 Solucion reto deportistas

In [ ]:
#esta clase muestra la solucion del profesor al reto.
deportistaPaises = deportistaOlimpicoRDD \
    .map(lambda l: [l[-1], l[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]]))

deportistaPaises.join(resultadoGanador).take(6)


#Clase 12 Operaciones Numéricas
##En esta clase hacemos el encoding de los valores de las medallas y obtenemos los puntos que tiene cada país.
Acciones de Conteo sobre RDD

```
# Tiene formato de código
```



In [ ]:
!ls ./sample_data/files_curso_spark/

In [ ]:
#Almacenamos el puntaje por medalla
valoresMedallas = {'Gold'  :7,
                   'Silver':5,
                   'Bronze':4}

In [ ]:
paisesMedallas = deportistaPaises.join(resultadoGanador)



In [ ]:
paisesMedallas = paisesMedallas \
    .map(lambda x: (x[1][0][-1], valoresMedallas[x[1][1] ] ) )

In [ ]:
paisesMedallas.take(5)

In [ ]:
from operator import add

conclusion = paisesMedallas.reduceByKey((add)) \
    .sortBy(lambda x:x[1], ascending=False)

In [ ]:
conclusion.take(10)

Clase 13 Creación de DataFrames

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.types import Row

from pyspark.sql import SQLContext





In [ ]:
#spark = SparkContext(master='local', appName="DataFrames")
sqlContext = SQLContext(spark)




In [ ]:
!ls ./sample_data/files_curso_spark/

In [ ]:
!head -n 10 ./sample_data/files_curso_spark/juegos.csv

In [ ]:
spark

In [ ]:
# Paso 1 crear un dataFrame es crearun Schema como una base de datos
path = '/content/sample_data/files_curso_spark/'
juegoSchema = StructType([
     StructField('juego_id', IntegerType(), False),
     StructField('anio', StringType(), False),
     StructField('temporada', StringType(), False),
     StructField('ciudad', StringType(), False),
    ])
# Paso 2 Crea Data Frame con la Lectura del Schema 
juegoDF = sqlContext.read.schema(juegoSchema) \
          .option('header', 'true').csv(path+'resultados.csv')

In [ ]:
juegoDF.show(4)

Clase 14 Inferencia de tipos de datos

In [ ]:
deportistaOlimpicoRDD.take(5)

In [ ]:
# Retiramos encabezados
def eliminarEncabezado(indice, iterador):
  return iter(list(iterador)[1:])

In [ ]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.\
mapPartitionsWithIndex(eliminarEncabezado)

In [ ]:
deportistaOlimpicoRDD.take(5)

In [ ]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6]),
))

In [ ]:
deportistaOlimpicoRDD.take(5)

In [ ]:
schema_deportistaOlimpico = StructType([
    StructField('deportista_id', IntegerType(),False),
    StructField('nombre', StringType(),False),
    StructField('genero', IntegerType(),False),
    StructField('edad', IntegerType(),False),
    StructField('altura', IntegerType(),False),
    StructField('peso', FloatType(),False),
    StructField('equipo_id', IntegerType(),False)
]    
)

In [ ]:
deportistaDF =  sqlContext.createDataFrame(deportistaOlimpicoRDD, schema_deportistaOlimpico)

In [ ]:
deportistaDF.show(5)

Retos crear los demas DataFrames testantes a partir de los RDD

In [ ]:
# Países  equipos, sc???
paisesRDD = sc.textFile(path+'paises.csv') \
.map(lambda line: line.split(",")) \
.mapPartitionsWithIndex(eliminarEncabezado) \
.map(lambda l:(
    int(l[0]),
    l[1],
    l[2]
))

schema = StructType([
    StructField('pais_id', IntegerType(),False),
    StructField('equipo', StringType(),False),
    StructField('sigla', StringType(),False)

]    
)

paisesDF =  sqlContext.createDataFrame(paisesRDD, schema)
paisesDF.show(5)

In [ ]:
resultados_schema = StructType([
    StructField("resultado_id", IntegerType(), False),
    StructField("medalla", StringType(), False),
    StructField("deportista_id", IntegerType(), False),
    StructField("juego_id", IntegerType(), False),
    StructField("evento_id", IntegerType(), False)
])

resultadosDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(resultados_schema).\
            load(path+"resultados.csv")

resultadosDF.show(4)

In [ ]:
evento_schema = StructType([
    StructField("evento_id", IntegerType(), False),
    StructField("evento", StringType(), False), 
    StructField("deporte_id", IntegerType(), False)
])

eventoDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(evento_schema).\
        load(path+"evento.csv")

eventoDF.show(4)

In [ ]:
deporte_schema = StructType([
          StructField("deporte_id",IntegerType(),False),
          StructField("deporte", StringType(),False)
])

deporteDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(deporte_schema).\
        load(path+"deporte.csv")

deporteDF.show(4)

In [ ]:
# juegos

juegos_schema = StructType([
    StructField('nombre_juego', StringType(),False),
    StructField('annio', IntegerType(),False),
    StructField('temporada', StringType(),False),
    StructField('ciudad', StringType(),False)
]    
)

juegosDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(juegos_schema).\
        load(path+"juegos.csv")

juegosDF.show(4)


 Clase 15 Operaciones sobre DF.
En esta clase aprendemos como obtener el schema de un DataFrame, a renombrar columnas y select().

In [ ]:
# Ver el esquema
deporteDF.printSchema()

In [ ]:
# Ver el esquema
deportistaDF.printSchema()

In [ ]:
#Renombrar columnas
deportistaOlimpicoDF = deportistaDF \
.withColumnRenamed("genero", 'sexo') \
.drop('altura')


In [ ]:
deportistaOlimpicoDF.printSchema()

In [ ]:
# Usando el rey SELECT
from pyspark.sql.functions import *
deportistaOlimpicoDF = deportistaOlimpicoDF.select("deportista_id","nombre",
 col("edad").alias("edadAlJugar"),
 "equipo_id"
 )

In [ ]:
deportistaOlimpicoDF.show(5)

In [ ]:
# Usando filter
deportistaOlimpicoDF = deportistaOlimpicoDF.filter(
    (deportistaOlimpicoDF.edadAlJugar != 0)
)

In [ ]:
deportistaOlimpicoDF.sort("edadAlJugar").show()

Clase 16 Agrupaciones y operaciones join sobre DF


In [ ]:
#vizualizamos los esquemas de los DF
deportistaOlimpicoDF.printSchema()

In [ ]:
resultadosDF.printSchema()

In [ ]:
juegosDF.printSchema()

In [ ]:
deporteDF.printSchema()

In [ ]:
paisesDF.printSchema()

In [ ]:
#JOINS
deportistaOlimpicoDF \
.join(
    resultadosDF,
    deportistaOlimpicoDF.deportista_id == resultadosDF.deportista_id,
    "left"
) \
.join(
    juegoDF,
    juegoDF.juego_id == resultadosDF.juego_id,
) \
.join(
    eventoDF,
    eventoDF.evento_id == resultadosDF.evento_id,
    "left"
) \
.select(deportistaOlimpicoDF.nombre,
        "edadAlJugar",
        "medalla",
        col("anio").alias("annio de juego"),
        deportistaOlimpicoDF.nombre.alias("Nombre disciplina")   
).show()



In [ ]:
#Reto hacer un JOIN con todas las medallas ganadoras
#unidas con el país y el equipo al que pertenecen estas medallas
resultadosDF \
.join(
    deportistaOlimpicoDF,
   resultadosDF.deportista_id == deportistaOlimpicoDF.deportista_id,
    "left"
) \
.join(
   paisesDF,
    deportistaOlimpicoDF.equipo_id == paisesDF.pais_id ,
    "left"
) \
.select("medalla",
        paisesDF.equipo,
       paisesDF.sigla
) \
.where (resultadosDF.medalla != "NA").show()




In [ ]:
# Clase 17 Solucion reto joins
resultadosDF.filter(resultadosDF.medalla != "NA") \
.join(
    deportistaOlimpicoDF,
     resultadosDF.deportista_id == deportistaOlimpicoDF.deportista_id,
     "left"     
) \
.join(
    paisesDF,
     deportistaOlimpicoDF.equipo_id == paisesDF.pais_id,
     "left"
) \
.select("medalla", paisesDF.equipo, paisesDF.sigla) \
.sort(col("sigla").desc()) \
.show()

In [ ]:
#Clase 18 Funciones de agrupación
#En esta clase revisamos cuales son las funciones de agrupación con que cuenta Spark.

In [ ]:
evento_schema = StructType([
    StructField('evento_id', IntegerType(),False),
    StructField('nombre', StringType(),False),
    StructField('deporte_id', IntegerType(),False)
]    
)

deportesOlimpicosDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(evento_schema).\
        load(path+"evento.csv")

deportesOlimpicosDF.show(4)

In [ ]:
#Clase 18 Funciones de agrupación
medallistaXanio = deportistaOlimpicoDF \
   .join (
       resultadosDF,
       deportistaOlimpicoDF.deportista_id == resultadosDF.deportista_id,
       "left"
         )\
   .join(
       juegoDF,
       juegoDF.juego_id == resultadosDF.juego_id,
       "left"
        )\
   .join(
     paisesDF,
     deportistaOlimpicoDF.equipo_id == paisesDF.pais_id,
     "left"
        )\
     .join(
         deportesOlimpicosDF,
         deportesOlimpicosDF.evento_id == resultadosDF.evento_id,
         "left"
        )\
      .join(
          deporteDF,
          deportesOlimpicosDF.deporte_id == deporteDF.deporte_id,
          "left"
          )\
       .select(
           "sigla",
           "anio",
           "medalla",
           deportesOlimpicosDF.nombre.alias("Nombre subdisciplina"),
           deporteDF.deporte.alias("Nombre disciplina"),
           deportistaOlimpicoDF.nombre
       )            


In [ ]:
medallistaXanio.show()

In [ ]:
#Descartamos deportistas sin medalla
medallistaXAnio2 = medallistaXanio.filter(resultadosDF.medalla != "NA")\
                                  .sort("anio")\
                                  .groupBy("sigla","anio","nombre subdisciplina")\
                                  .count() 
medallistaXAnio2.show()                                  

In [ ]:
medallistaXAnio2.printSchema()

In [ ]:
# .agg o agragate es la forma recomendada pos la documentación
medallistaXAnio2.groupBy("Sigla","anio")\
    .agg(sum("count").alias("Total de medallas"), \
     avg("count").alias("Medallas promedio")).show()

Clase 19 SQL

En esta clase registramos DataFrames como si fueran SQL y veremos las diferencias entre ejecutar SQL como si fuera Spark y la forma nativa.

In [ ]:
# registramos DataAframes como tables SQL
resultadosDF.registerTempTable("resultado")
deportistaOlimpicoDF.registerTempTable("deportista")
paisesDF.registerTempTable("paises")

In [ ]:
sqlContext.sql("SELECT * FROM deportista").show(5)

In [ ]:
#En términos generales el script de SQL es mas rápido que utilizar la sintaxis de Spark, 
#sin embargo el trade-off es que es mas costoso computacionalmente, esto lo podemos 
#observar en el gráfico de SparkUI, donde Catalyst nos ayuda.
sqlContext.sql("""
               SELECT medalla, equipo, sigla FROM resultado as r
               JOIN deportista as d
               ON r.deportista_id = d.deportista_id
               JOIN paises as p
               ON p.id_ = d.equipo_id
               WHERE medalla <> "NA"
               ORDER BY sigla DESC
              """).show(15)
#Recomendación, si vas a hacer cruces, joins o funciones de agregacion es mejor usar 
#el poder de SPARK y sus funciones nativas.

In [ ]:
spark

In [ ]:
!ls ./sample_data/files_curso_spark/

In [ ]:
!head -n 5 ./sample_data/files_curso_spark/deportista.csv

#Clase 21 UDF

#Una de las ventajas de las UDF es registrarlas para que #trabajen nativamente y ademas poder usarlas con las #consultas de SQL estándar

In [ ]:
DeportistaError_schema = StructType([
    StructField("deportista_id", StringType(), False),
    StructField("nombre", StringType(), False),
    StructField("genero", StringType(), False),
    StructField("edad", StringType(), False),
    StructField("altura", StringType(), False), 
    StructField("peso", StringType(), False),
    StructField("equipo_id", StringType(), False)
])

DeportistaErrorDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(DeportistaError_schema).\
            load(path+"deportistaError.csv")